# trying to operate batfish with these config files


# Import packages
%run startup.py
bf = Session(host="localhost")

In [1]:
# if you need to reinstall pybatfish
install_packages = False
if install_packages:
    !pip install pybatfish
    !pip install tqdm 


In [2]:
# Importing required libraries, setting up logging, and loading questions
import logging
import random  # noqa: F401
from typing import List, Optional  # noqa: F401

import pandas as pd
from IPython.display import display
from pandas.io.formats.style import Styler

from pybatfish.client.session import Session  # noqa: F401

# noinspection PyUnresolvedReferences
from pybatfish.datamodel import Edge, Interface  # noqa: F401
from pybatfish.datamodel.answer import TableAnswer
from pybatfish.datamodel.flow import HeaderConstraints, PathConstraints  # noqa: F401
from pybatfish.datamodel.route import BgpRoute  # noqa: F401
from pybatfish.util import get_html

# Configure all pybatfish loggers to use WARN level
logging.getLogger("pybatfish").setLevel(logging.WARN)

pd.set_option("display.max_colwidth", None)
pd.set_option("display.max_columns", None)
# Prevent rendering text between '$' as MathJax expressions
pd.set_option("display.html.use_mathjax", False)

# UUID for CSS styles used by pandas styler.
# Keeps our notebook HTML deterministic when displaying dataframes
_STYLE_UUID = "pybfstyle"


class MyStyler(Styler):
    """A custom styler for displaying DataFrames in HTML"""

    def __repr__(self):
        return repr(self.data)


def show(df):
    """
    Displays a dataframe as HTML table.

    Replaces newlines and double-spaces in the input with HTML markup, and
    left-aligns the text.
    """
    if isinstance(df, TableAnswer):
        df = df.frame()

    # workaround for Pandas bug in Python 2.7 for empty frames
    if not isinstance(df, pd.DataFrame) or df.size == 0:
        display(df)
        return
    display(
        MyStyler(df)
        .set_uuid(_STYLE_UUID)
        .format(get_html)
        .set_properties(**{"text-align": "left", "vertical-align": "top"})
    )


In [3]:
# Import packages

bf = Session(host="localhost")

In [4]:
# Assign a friendly name to your network and snapshot
NETWORK_NAME = "example_notwork"
SNAPSHOT_NAME = "example_snipshot"

# this is the path to the network. You need to spell it correctly
SNAPSHOT_PATH = "networks/elecnet"

# Now create the network and initialize the snapshot
bf.set_network(NETWORK_NAME)
bf.init_snapshot(SNAPSHOT_PATH, name=SNAPSHOT_NAME, overwrite=True)

'example_snipshot'

In [5]:
parse_status = bf.q.fileParseStatus().answer().frame()


In [6]:
for i in range(len(parse_status)):
    print(parse_status.iloc[i])
    print(' ')

File_Name       configs/adlcore-1.cfg
Status         PARTIALLY_UNRECOGNIZED
File_Format      CUMULUS_CONCATENATED
Nodes                   ['adlcore-1']
Name: 0, dtype: object
 
File_Name       configs/adlcore-2.cfg
Status         PARTIALLY_UNRECOGNIZED
File_Format      CUMULUS_CONCATENATED
Nodes                   ['adlcore-2']
Name: 1, dtype: object
 
File_Name       configs/bnecore-1.cfg
Status         PARTIALLY_UNRECOGNIZED
File_Format      CUMULUS_CONCATENATED
Nodes                   ['bnecore-1']
Name: 2, dtype: object
 
File_Name       configs/bnecore-2.cfg
Status         PARTIALLY_UNRECOGNIZED
File_Format      CUMULUS_CONCATENATED
Nodes                   ['bnecore-2']
Name: 3, dtype: object
 
File_Name          configs/brd-gw.cfg
Status         PARTIALLY_UNRECOGNIZED
File_Format      CUMULUS_CONCATENATED
Nodes                      ['brd-gw']
Name: 4, dtype: object
 
File_Name       configs/brwnodegw.cfg
Status         PARTIALLY_UNRECOGNIZED
File_Format      CUMULUS_CONCATENATED
N

In [7]:
# An example: use a filter on the returned dataframe to see which files failed to parse completely
parse_status[parse_status['Status'] != 'PASSED']  # change '!=' to '==' to get the files which passed


,File_Name,Status,File_Format,Nodes
0,configs/adlcore-1.cfg,PARTIALLY_UNRECOGNIZED,CUMULUS_CONCATENATED,['adlcore-1']
1,configs/adlcore-2.cfg,PARTIALLY_UNRECOGNIZED,CUMULUS_CONCATENATED,['adlcore-2']
2,configs/bnecore-1.cfg,PARTIALLY_UNRECOGNIZED,CUMULUS_CONCATENATED,['bnecore-1']
3,configs/bnecore-2.cfg,PARTIALLY_UNRECOGNIZED,CUMULUS_CONCATENATED,['bnecore-2']
4,configs/brd-gw.cfg,PARTIALLY_UNRECOGNIZED,CUMULUS_CONCATENATED,['brd-gw']
5,configs/brwnodegw.cfg,PARTIALLY_UNRECOGNIZED,CUMULUS_CONCATENATED,['brwnodegw']
6,configs/byo-gw.cfg,PARTIALLY_UNRECOGNIZED,CUMULUS_CONCATENATED,['byo-gw']
7,configs/cbpdnodegw.cfg,PARTIALLY_UNRECOGNIZED,CUMULUS_CONCATENATED,['cbpdnodegw']
8,configs/cbrcore-1.cfg,PARTIALLY_UNRECOGNIZED,CUMULUS_CONCATENATED,['cbrcore-1']
9,configs/cbrcore-2.cfg,PARTIALLY_UNRECOGNIZED,CUMULUS_CONCATENATED,['cbrcore-2']


In [8]:
# An example: use a filter on the returned dataframe to see which files parse completely
parse_status[parse_status['Status'] == 'PASSED']  # change '!=' to '==' to get the files which passed


,File_Name,Status,File_Format,Nodes
56,hosts/adlcore-1.json,PASSED,HOST,['adlcore-1']
57,hosts/adlcore-2.json,PASSED,HOST,['adlcore-2']
58,hosts/bnecore-1.json,PASSED,HOST,['bnecore-1']
59,hosts/bnecore-2.json,PASSED,HOST,['bnecore-2']
60,hosts/brd-gw.json,PASSED,HOST,['brd-gw']
...,...,...,...,...
337,iptables/wwanode0.iptables,PASSED,IPTABLES,['iptables/wwanode0.iptables']
338,iptables/wwanode1.iptables,PASSED,IPTABLES,['iptables/wwanode1.iptables']
339,iptables/wwanode2.iptables,PASSED,IPTABLES,['iptables/wwanode2.iptables']
340,iptables/wwanodegw.iptables,PASSED,IPTABLES,['iptables/wwanodegw.iptables']


In [9]:
# View details if some of the files were not parsed completely
for i in range(220, 240):
    print(bf.q.parseWarning().answer().frame().iloc[i])
    print(' ')

Filename                                              configs/cmtnodegw.cfg
Line                                                                     57
Text                                                ip ospf dead-interval 6
Parser_Context    [siip_ospf si_ip s_interface statement frr_configuration]
Comment                                         This syntax is unrecognized
Name: 220, dtype: object
 
Filename                                              configs/cmtnodegw.cfg
Line                                                                     58
Text                                               ip ospf hello-interval 2
Parser_Context    [siip_ospf si_ip s_interface statement frr_configuration]
Comment                                         This syntax is unrecognized
Name: 221, dtype: object
 
Filename                              configs/cmtnodegw.cfg
Line                                                     60
Text                                                   exit
Parser

In [10]:
interface_properties = bf.q.interfaceProperties().answer().frame()

In [11]:
interface_properties

,Interface,Access_VLAN,Active,Admin_Up,All_Prefixes,Allowed_VLANs,Auto_State_VLAN,Bandwidth,Blacklisted,Channel_Group,Channel_Group_Members,DHCP_Relay_Addresses,Declared_Names,Description,Encapsulation_VLAN,HSRP_Groups,HSRP_Version,Inactive_Reason,Incoming_Filter_Name,MLAG_ID,MTU,Native_VLAN,Outgoing_Filter_Name,PBR_Policy_Name,Primary_Address,Primary_Network,Proxy_ARP,Rip_Enabled,Rip_Passive,Spanning_Tree_Portfast,Speed,Switchport,Switchport_Mode,Switchport_Trunk_Encapsulation,VRF,VRRP_Groups,Zone_Name
0,adlcore-1[eth0],None,True,True,['10.21.9.193/30'],,True,10000000000.0,False,None,[],[],[],None,None,[],None,,None,None,1500,None,filter::FORWARD,None,10.21.9.193/30,10.21.9.192/30,False,False,False,False,None,False,NONE,DOT1Q,default,[],None
1,adlcore-1[eth1],None,True,True,['10.22.27.115/29'],,True,10000000000.0,False,None,[],[],[],None,None,[],None,,None,None,1500,None,filter::FORWARD,None,10.22.27.115/29,10.22.27.112/29,False,False,False,False,None,False,NONE,DOT1Q,default,[],None
2,adlcore-1[eth2],None,True,True,['10.22.11.73/29'],,True,10000000000.0,False,None,[],[],[],None,None,[],None,,None,None,1500,None,filter::FORWARD,None,10.22.11.73/29,10.22.11.72/29,False,False,False,False,None,False,NONE,DOT1Q,default,[],None
3,adlcore-1[eth3],None,True,True,['10.10.22.25/30'],,True,10000000000.0,False,None,[],[],[],None,None,[],None,,None,None,1500,None,filter::FORWARD,None,10.10.22.25/30,10.10.22.24/30,False,False,False,False,None,False,NONE,DOT1Q,default,[],None
4,adlcore-1[eth4],None,True,True,['10.10.10.198/30'],,True,10000000000.0,False,None,[],[],[],None,None,[],None,,None,None,1500,None,filter::FORWARD,None,10.10.10.198/30,10.10.10.196/30,False,False,False,False,None,False,NONE,DOT1Q,default,[],None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
401,wwanodegw[lo],None,True,True,['127.0.0.1/8'],,True,8000000000.0,None,None,[],[],[],None,None,[],None,,None,None,65536,None,filter::FORWARD,None,127.0.0.1/8,127.0.0.0/8,False,False,False,False,None,False,NONE,DOT1Q,default,[],None
402,yrv-gw[eth0],None,True,True,['10.22.217.196/29'],,True,10000000000.0,False,None,[],[],[],None,None,[],None,,None,None,1500,None,filter::FORWARD,None,10.22.217.196/29,10.22.217.192/29,False,False,False,False,None,False,NONE,DOT1Q,default,[],None
403,yrv-gw[eth1],None,True,True,['10.22.221.181/29'],,True,10000000000.0,False,None,[],[],[],None,None,[],None,,None,None,1500,None,filter::FORWARD,None,10.22.221.181/29,10.22.221.176/29,False,False,False,False,None,False,NONE,DOT1Q,default,[],None
404,yrv-gw[eth2],None,True,True,['10.23.208.165/29'],,True,10000000000.0,False,None,[],[],[],None,None,[],None,,None,None,1500,None,filter::FORWARD,None,10.23.208.165/29,10.23.208.160/29,False,False,False,False,None,False,NONE,DOT1Q,default,[],None


In [12]:
node_properties  = bf.q.nodeProperties(properties="Domain_Name,NTP_Servers,Interfaces").answer().frame()
node_properties

,Node,Domain_Name,Interfaces,NTP_Servers
0,nkr-gw,None,"['eth0', 'eth1', 'eth2', 'lo']",[]
1,bnecore-2,None,"['eth0', 'eth1', 'eth2', 'eth3', 'eth4', 'eth5', 'lo']",[]
2,dwncore-2,None,"['eth0', 'eth1', 'eth2', 'eth3', 'eth4', 'eth5', 'lo']",[]
3,sydcore-2,None,"['eth0', 'eth1', 'eth2', 'eth3', 'eth4', 'eth5', 'lo']",[]
4,sydcore-1,None,"['eth0', 'eth1', 'eth2', 'eth3', 'eth4', 'eth5', 'lo']",[]
...,...,...,...,...
138,dwncore-1,None,"['eth0', 'eth1', 'eth2', 'eth3', 'eth4', 'eth5', 'lo']",[]
139,meeknodegw,None,"['eth0', 'eth1', 'eth2', 'eth3', 'lo']",[]
140,nmnnode1,None,['eth0'],[]
141,hoenode1,None,['eth0'],[]


In [13]:
# Fetch specific properties of Loopback interfaces
bf.q.interfaceProperties(interfaces="/eth/", properties="Bandwidth,VRF,Primary_Address").answer().frame()

,Interface,Bandwidth,Primary_Address,VRF
0,adlcore-1[eth0],10000000000.0,10.21.9.193/30,default
1,adlcore-1[eth1],10000000000.0,10.22.27.115/29,default
2,adlcore-1[eth2],10000000000.0,10.22.11.73/29,default
3,adlcore-1[eth3],10000000000.0,10.10.22.25/30,default
4,adlcore-1[eth4],10000000000.0,10.10.10.198/30,default
...,...,...,...,...
345,wwanodegw[eth2],10000000000.0,10.25.165.206/25,default
346,wwanodegw[eth3],10000000000.0,10.25.160.146/24,default
347,yrv-gw[eth0],10000000000.0,10.22.217.196/29,default
348,yrv-gw[eth1],10000000000.0,10.22.221.181/29,default


In [14]:
# Get layer 3 edges
for i in node_properties.Node:
    print(i)
    print(bf.q.edges(nodes=i).answer().frame())
    print(' ')

nkr-gw
      Interface               IPs Remote_Interface        Remote_IPs
0  nkr-gw[eth0]  ['10.22.93.123']  dwncore-1[eth1]  ['10.22.93.121']
1  nkr-gw[eth1]  ['10.23.91.121']    eme-nkr[eth0]  ['10.23.91.126']
2  nkr-gw[eth1]  ['10.23.91.121']    ess-nkr[eth0]  ['10.23.91.124']
3  nkr-gw[eth2]  ['10.22.83.189']  dwncore-2[eth4]  ['10.22.83.186']
 
bnecore-2
         Interface                IPs Remote_Interface         Remote_IPs
0  bnecore-2[eth0]  ['10.22.126.109']     sun-gw[eth1]  ['10.22.126.107']
1  bnecore-2[eth1]   ['10.22.126.89']     byo-gw[eth0]   ['10.22.126.92']
2  bnecore-2[eth2]    ['10.20.99.61']     mkyagg[eth0]    ['10.20.99.62']
3  bnecore-2[eth3]   ['10.10.97.169']  sydcore-2[eth0]   ['10.10.97.170']
4  bnecore-2[eth4]    ['10.10.67.10']  dwncore-2[eth0]     ['10.10.67.9']
5  bnecore-2[eth5]  ['10.21.103.146']  bnecore-1[eth1]  ['10.21.103.145']
 
dwncore-2
         Interface               IPs Remote_Interface        Remote_IPs
0  dwncore-2[eth0]    ['10.10.67.9

In [15]:
# List references to undefined structures
bf.q.undefinedReferences().answer().frame()

,File_Name,Struct_Type,Ref_Name,Context,Lines


In [16]:
# Get all edges 
bf.q.edges().answer().frame()

,Interface,IPs,Remote_Interface,Remote_IPs
0,adlcore-1[eth0],['10.21.9.193'],adlcore-2[eth2],['10.21.9.194']
1,adlcore-1[eth1],['10.22.27.115'],tor-gw[eth2],['10.22.27.116']
2,adlcore-1[eth2],['10.22.11.73'],edn-gw[eth1],['10.22.11.75']
3,adlcore-1[eth3],['10.10.22.25'],percore-1[eth3],['10.10.22.26']
4,adlcore-1[eth4],['10.10.10.198'],melcore-1[eth2],['10.10.10.197']
...,...,...,...,...
387,wwanodegw[eth3],['10.25.160.146'],wwanode1[eth0],['10.25.160.57']
388,yrv-gw[eth0],['10.22.217.196'],melcore-2[eth3],['10.22.217.197']
389,yrv-gw[eth1],['10.22.221.181'],melcore-1[eth4],['10.22.221.180']
390,yrv-gw[eth2],['10.23.208.165'],eme-yrv[eth0],['10.23.208.164']


In [17]:
# Do a traceroute from host1 to 1.0.2.2
tr_frame = bf.q.traceroute(startLocation="adlcore-1", headers=HeaderConstraints(dstIps="10.25.15.38")).answer().frame()

# Display results using customizations to handle large string values

show(tr_frame)

,Flow,Traces,TraceCount
0,Start Location: adlcore-1Src IP: 10.10.10.198Src Port: 49152Dst IP: 10.25.15.38Dst Port: 33434IP Protocol: UDP,"ACCEPTED1. node: adlcore-1 ORIGINATED(default) FORWARDED(Forwarded out interface: eth3 with resolved next-hop IP: 10.10.22.26, Routes: [ospf (Network: 10.25.15.0/25, Next Hop: interface eth3 ip 10.10.22.26)]) PERMITTED(filter::FORWARD (EGRESS_FILTER)) TRANSMITTED(eth3)2. node: percore-1 RECEIVED(eth3) FORWARDED(Forwarded out interface: eth6 with resolved next-hop IP: 10.20.38.46, Routes: [ospf (Network: 10.25.15.0/25, Next Hop: interface eth6 ip 10.20.38.46)]) PERMITTED(filter::FORWARD (EGRESS_FILTER)) TRANSMITTED(eth6)3. node: krthagg RECEIVED(eth1) FORWARDED(Forwarded out interface: eth2 with resolved next-hop IP: 10.25.205.112, Routes: [ospf (Network: 10.25.15.0/25, Next Hop: interface eth2 ip 10.25.205.112)]) PERMITTED(filter::FORWARD (EGRESS_FILTER)) TRANSMITTED(eth2)4. node: brwnodegw RECEIVED(eth0) ACCEPTED(eth1)",1
1,Start Location: adlcore-1Src IP: 10.10.22.25Src Port: 49152Dst IP: 10.25.15.38Dst Port: 33434IP Protocol: UDP,"ACCEPTED1. node: adlcore-1 ORIGINATED(default) FORWARDED(Forwarded out interface: eth3 with resolved next-hop IP: 10.10.22.26, Routes: [ospf (Network: 10.25.15.0/25, Next Hop: interface eth3 ip 10.10.22.26)]) PERMITTED(filter::FORWARD (EGRESS_FILTER)) TRANSMITTED(eth3)2. node: percore-1 RECEIVED(eth3) FORWARDED(Forwarded out interface: eth6 with resolved next-hop IP: 10.20.38.46, Routes: [ospf (Network: 10.25.15.0/25, Next Hop: interface eth6 ip 10.20.38.46)]) PERMITTED(filter::FORWARD (EGRESS_FILTER)) TRANSMITTED(eth6)3. node: krthagg RECEIVED(eth1) FORWARDED(Forwarded out interface: eth2 with resolved next-hop IP: 10.25.205.112, Routes: [ospf (Network: 10.25.15.0/25, Next Hop: interface eth2 ip 10.25.205.112)]) PERMITTED(filter::FORWARD (EGRESS_FILTER)) TRANSMITTED(eth2)4. node: brwnodegw RECEIVED(eth0) ACCEPTED(eth1)",1
2,Start Location: adlcore-1Src IP: 10.20.12.133Src Port: 49152Dst IP: 10.25.15.38Dst Port: 33434IP Protocol: UDP,"ACCEPTED1. node: adlcore-1 ORIGINATED(default) FORWARDED(Forwarded out interface: eth3 with resolved next-hop IP: 10.10.22.26, Routes: [ospf (Network: 10.25.15.0/25, Next Hop: interface eth3 ip 10.10.22.26)]) PERMITTED(filter::FORWARD (EGRESS_FILTER)) TRANSMITTED(eth3)2. node: percore-1 RECEIVED(eth3) FORWARDED(Forwarded out interface: eth6 with resolved next-hop IP: 10.20.38.46, Routes: [ospf (Network: 10.25.15.0/25, Next Hop: interface eth6 ip 10.20.38.46)]) PERMITTED(filter::FORWARD (EGRESS_FILTER)) TRANSMITTED(eth6)3. node: krthagg RECEIVED(eth1) FORWARDED(Forwarded out interface: eth2 with resolved next-hop IP: 10.25.205.112, Routes: [ospf (Network: 10.25.15.0/25, Next Hop: interface eth2 ip 10.25.205.112)]) PERMITTED(filter::FORWARD (EGRESS_FILTER)) TRANSMITTED(eth2)4. node: brwnodegw RECEIVED(eth0) ACCEPTED(eth1)",1
3,Start Location: adlcore-1Src IP: 10.21.9.193Src Port: 49152Dst IP: 10.25.15.38Dst Port: 33434IP Protocol: UDP,"ACCEPTED1. node: adlcore-1 ORIGINATED(default) FORWARDED(Forwarded out interface: eth3 with resolved next-hop IP: 10.10.22.26, Routes: [ospf (Network: 10.25.15.0/25, Next Hop: interface eth3 ip 10.10.22.26)]) PERMITTED(filter::FORWARD (EGRESS_FILTER)) TRANSMITTED(eth3)2. node: percore-1 RECEIVED(eth3) FORWARDED(Forwarded out interface: eth6 with resolved next-hop IP: 10.20.38.46, Routes: [ospf (Network: 10.25.15.0/25, Next Hop: interface eth6 ip 10.20.38.46)]) PERMITTED(filter::FORWARD (EGRESS_FILTER)) TRANSMITTED(eth6)3. node: krthagg RECEIVED(eth1) FORWARDED(Forwarded out interface: eth2 with resolved next-hop IP: 10.25.205.112, Routes: [ospf (Network: 10.25.15.0/25, Next Hop: interface eth2 ip 10.25.205.112)]) PERMITTED(filter::FORWARD (EGRESS_FILTER)) TRANSMITTED(eth2)4. node: brwnodegw RECEIVED(eth0) ACCEPTED(eth1)",1
4,Start Location: adlcore-1Src IP: 10.22.11.73Src Port: 49152Dst IP: 10.25.15.38Dst Port: 33434IP Protocol: UDP,"ACCEPTED1. node: adlcore-1 ORIGINATED(default) FORWARDED(Forwarded ou

In [18]:
# Fetch the routing table of all VRFs on all nodes in the snapshot
routes_all = bf.q.routes().answer().frame()

In [19]:
routes_all

,Node,VRF,Network,Next_Hop,Next_Hop_IP,Next_Hop_Interface,Protocol,Metric,Admin_Distance,Tag
0,adlcore-1,default,10.10.2.96/30,interface eth0 ip 10.21.9.194,10.21.9.194,eth0,ospf,20,110,None
1,adlcore-1,default,10.10.9.204/30,interface eth0 ip 10.21.9.194,10.21.9.194,eth0,ospf,20,110,None
2,adlcore-1,default,10.10.10.196/30,interface eth4,AUTO/NONE(-1l),eth4,connected,0,0,None
3,adlcore-1,default,10.10.22.24/30,interface eth3,AUTO/NONE(-1l),eth3,connected,0,0,None
4,adlcore-1,default,10.10.45.128/30,interface eth3 ip 10.10.22.26,10.10.22.26,eth3,ospf,20,110,None
...,...,...,...,...,...,...,...,...,...,...
12541,yrv-gw,default,10.25.243.0/25,interface eth0 ip 10.22.217.197,10.22.217.197,eth0,ospf,30,110,None
12542,yrv-gw,default,10.25.243.0/25,interface eth1 ip 10.22.221.180,10.22.221.180,eth1,ospf,30,110,None
12543,yrv-gw,default,10.25.250.0/25,interface eth0 ip 10.22.217.197,10.22.217.197,eth0,ospf,30,110,None
12544,yrv-gw,default,10.25.250.0/25,interface eth1 ip 10.22.221.180,10.22.221.180,eth1,ospf,30,110,None


In [99]:
# code that runs traceroutes from every node to every (relevant) IP address
interface_properties = bf.q.interfaceProperties(interfaces="/eth/", properties="Bandwidth,VRF,Primary_Address").answer().frame()
node_properties  = bf.q.nodeProperties(properties="Domain_Name,NTP_Servers,Interfaces").answer().frame()

traceroute_list = []
counter = 0

number_of_trials = 100

while counter < number_of_trials:
    node_name = random.choice(node_properties.Node)
    IP_address = random.choice(interface_properties.Primary_Address)
    traceroute_output = bf.q.traceroute(startLocation=node_name, headers=HeaderConstraints(dstIps=IP_address)).answer().frame()
    traceroute_list.append(traceroute_output)
    print(traceroute_output.Traces[0][0].disposition)
    counter +=1




DENIED_OUT
DENIED_OUT
EXITS_NETWORK
EXITS_NETWORK
DENIED_OUT
NEIGHBOR_UNREACHABLE
EXITS_NETWORK
NEIGHBOR_UNREACHABLE
DENIED_OUT
DENIED_OUT
EXITS_NETWORK
DENIED_OUT
DENIED_OUT
DENIED_OUT
EXITS_NETWORK
EXITS_NETWORK
DENIED_OUT
DENIED_OUT
EXITS_NETWORK
NEIGHBOR_UNREACHABLE
EXITS_NETWORK
EXITS_NETWORK
DENIED_OUT
EXITS_NETWORK
EXITS_NETWORK
DENIED_OUT
NEIGHBOR_UNREACHABLE
NEIGHBOR_UNREACHABLE
DENIED_OUT
NEIGHBOR_UNREACHABLE
DENIED_OUT
DENIED_OUT
EXITS_NETWORK
NEIGHBOR_UNREACHABLE
EXITS_NETWORK
EXITS_NETWORK
NEIGHBOR_UNREACHABLE
DENIED_OUT
DENIED_OUT
DENIED_OUT
DENIED_OUT
DENIED_OUT
EXITS_NETWORK
EXITS_NETWORK
EXITS_NETWORK
EXITS_NETWORK
EXITS_NETWORK
EXITS_NETWORK
DENIED_OUT
EXITS_NETWORK
NEIGHBOR_UNREACHABLE
NEIGHBOR_UNREACHABLE
EXITS_NETWORK
DENIED_OUT
DENIED_OUT
DENIED_OUT
DENIED_OUT
EXITS_NETWORK
NEIGHBOR_UNREACHABLE
EXITS_NETWORK
DENIED_OUT
EXITS_NETWORK
EXITS_NETWORK
NEIGHBOR_UNREACHABLE
DENIED_OUT
EXITS_NETWORK
NEIGHBOR_UNREACHABLE
EXITS_NETWORK
EXITS_NETWORK
DENIED_OUT
EXITS_NETWORK

In [106]:
for i in range(9):
    print(traceroute_list[i].Traces[0][0].disposition)

DENIED_OUT
DENIED_OUT
EXITS_NETWORK
EXITS_NETWORK
DENIED_OUT
NEIGHBOR_UNREACHABLE
EXITS_NETWORK
NEIGHBOR_UNREACHABLE
DENIED_OUT


In [ ]:
traceroute_list[]

,Flow,Traces,TraceCount
0,start=ess-sun [10.23.115.34:49152->10.24.120.0:33434 UDP],"[((ORIGINATED(default), FORWARDED(Forwarded out interface: eth0 with resolved next-hop IP: 10.23.115.37, Routes: [static (Network: 0.0.0.0/0, Next Hop: interface eth0 ip 10.23.115.37)]), PERMITTED(filter::OUTPUT (EGRESS_FILTER)), TRANSMITTED(eth0)), (RECEIVED(eth0), FORWARDED(Forwarded out interface: eth1 with resolved next-hop IP: 10.22.126.109, Routes: [ospf (Network: 10.24.120.0/24, Next Hop: interface eth1 ip 10.22.126.109)]), PERMITTED(filter::FORWARD (EGRESS_FILTER)), TRANSMITTED(eth1)), (RECEIVED(eth0), FORWARDED(Forwarded out interface: eth2 with resolved next-hop IP: 10.20.99.62, Routes: [ospf (Network: 10.24.120.0/24, Next Hop: interface eth2 ip 10.20.99.62)]), PERMITTED(filter::FORWARD (EGRESS_FILTER)), TRANSMITTED(eth2)), (RECEIVED(eth0), FORWARDED(Forwarded out interface: eth5, Routes: [connected (Network: 10.24.120.0/24, Next Hop: interface eth5)]), PERMITTED(filter::FORWARD (EGRESS_FILTER)), TRANSMITTED(eth5), EXITS_NETWORK(Output Interface: eth5, Resolved Next Hop IP: 10.24.120.0))), ((ORIGINATED(default), FORWARDED(Forwarded out interface: eth0 with resolved next-hop IP: 10.23.115.37, Routes: [static (Network: 0.0.0.0/0, Next Hop: interface eth0 ip 10.23.115.37)]), PERMITTED(filter::OUTPUT (EGRESS_FILTER)), TRANSMITTED(eth0)), (RECEIVED(eth0), FORWARDED(Forwarded out interface: eth2 with resolved next-hop IP: 10.22.115.230, Routes: [ospf (Network: 10.24.120.0/24, Next Hop: interface eth2 ip 10.22.115.230)]), PERMITTED(filter::FORWARD (EGRESS_FILTER)), TRANSMITTED(eth2)), (RECEIVED(eth0), FORWARDED(Forwarded out interface: eth5 with resolved next-hop IP: 10.20.98.65, Routes: [ospf (Network: 10.24.120.0/24, Next Hop: interface eth5 ip 10.20.98.65)]), PERMITTED(filter::FORWARD (EGRESS_FILTER)), TRANSMITTED(eth5)), (RECEIVED(eth1), FORWARDED(Forwarded out interface: eth5, Routes: [connected (Network: 10.24.120.0/24, Next Hop: interface eth5)]), PERMITTED(filter::FORWARD (EGRESS_FILTER)), TRANSMITTED(eth5), EXITS_NETWORK(Output Interface: eth5, Resolved Next Hop IP: 10.24.120.0)))]",2
